NETTOYAGE / FILTRAGE

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Name Basics

'''
Colonnes à supprimer : Aucune
Lignes à supprimer:
    - knownfortitle = /N
    - primaryprofession = /N
'''

In [ ]:
# Title Akas

'''
Colonnes à supprimer : 
Lignes à supprimer: 
    - Faire un groupby par ID de film
    - A voir plus tard pour filtrer avec un contains sur language et/ou région
'''

In [ ]:
# Title Basics

'''
Colonnes à supprimer : 
Lignes à supprimer: 
    - Titletype = movie
    - A voir pour filtrer par années par la suite
    - A voir pour filtrer selon durée des films (trop courts ou trop longs)
    - isAdult = 1 et genre = adult (uniquement)
    - Vérifier classification de Basic instinct

'''

In [ ]:
# Title Crew

'''
RAS
'''

In [ ]:
# Title Episodes

'''
A voir : concatener numberEpisode et numberSeason
'''

In [ ]:
# Title Principals

'''
Colonnes à supprimer : job, character
'''

In [ ]:
# Title Ratings

'''
RAS
'''

In [ ]:
link = "./BD/tmdb_full.csv"
df_tmdb_full = pd.read_csv(link)

df_tmdb_full['fr?'] = df_tmdb_full['spoken_languages'].apply(lambda x: 'fr' in x)
df_fr = df_tmdb_full[df_tmdb_full['fr?']]
link = "./BD/title.akas.tsv.gz"
df_title_akas = pd.read_csv(link, compression = 'gzip', sep = '\t')
df_title_akas = df_title_akas[(df_title_akas['region'] == 'FR') | (df_title_akas['language'] == 'fr')]
df_TMDB_AKAS = pd.merge(df_fr,
                        df_title_akas,
                        how = 'left',
                        left_on = df_fr['imdb_id'],
                        right_on = df_title_akas['titleId'])

df_title_basics = pd.read_csv("BD/title.basics.tsv.gz", compression = 'gzip', sep = '\t')
df_TMDB_AKAS = df_TMDB_AKAS.drop(['key_0'], axis = 1)
df = pd.merge(df_TMDB_AKAS,
              df_title_basics,
              how = 'left',
              left_on = df_TMDB_AKAS['imdb_id'],
              right_on = df_title_basics['tconst'])
df_title_crew = pd.read_csv("BD/title.crew.tsv.gz", compression = 'gzip', sep = '\t')
df = df.drop(['key_0'], axis = 1)
df = pd.merge(df,
              df_title_crew,
              how = 'left',
              left_on = df['imdb_id'],
              right_on = df_title_crew['tconst'])
df_rate = pd.read_csv("BD/title.ratings.tsv.gz", compression = 'gzip', sep = '\t')
df = df.drop(['key_0'], axis = 1)
df = pd.merge(df,
              df_rate,
              how = 'left',
              left_on = df['imdb_id'],
              right_on = df_rate['tconst'])

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29164 entries, 0 to 29163
Data columns (total 50 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   key_0                         29164 non-null  object 
 1   adult                         29164 non-null  bool   
 2   backdrop_path                 19416 non-null  object 
 3   budget                        29164 non-null  int64  
 4   genres_x                      29164 non-null  object 
 5   homepage                      3446 non-null   object 
 6   id                            29164 non-null  int64  
 7   imdb_id                       29164 non-null  object 
 8   original_language             29164 non-null  object 
 9   original_title                29164 non-null  object 
 10  overview                      26885 non-null  object 
 11  popularity                    29164 non-null  float64
 12  poster_path                   27405 non-null  object 
 13  p

In [ ]:
# L’identification des acteurs les plus présents et les périodes associées 
# Dans la table title.principals.tsv.gz on récupère toutes les lignes category == 'actor' 
# On récupère de l'autre côté la table title.basics.tsv.gz avec que les films

# On charge la bd title.principals et on remplace les \N par NaN
df_acteur = pd.read_csv('BD/title.principals.tsv.gz', compression = 'gzip', sep = '\t')
df_acteur = df_acteur[df_acteur['category'] == 'actor']
df_acteur.replace('\\N', np.nan , inplace = True)
    

C:\Users\lepor\AppData\Local\Temp\ipykernel_7444\3120977640.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_acteur.replace('\\N', np.nan , inplace = True)


In [ ]:
# On charge la bd title.basics et on remplace les \N par NaN

df_film = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')
df_film = df_film[df_film['titleType'] == 'movie']
df_film['startYear'].replace('\\N', np.nan, inplace = True)

C:\Users\lepor\AppData\Local\Temp\ipykernel_7444\1939738505.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_film = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')
C:\Users\lepor\AppData\Local\Temp\ipykernel_7444\1939738505.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_film['startYear'].replace('\\N', np.nan, inplace = True)


In [ ]:
# On merge les deux df pour relier chaque acteur à ses films 

df_1 = pd.merge(df_acteur,
                df_film,
                how = 'inner',
                on = 'tconst') 

,tconst,ordering,nconst,category,job,characters,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000009,2,nm0183823,actor,NaN,"[""Mr. Hamilton""]",movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
1,tt0000009,3,nm1309758,actor,NaN,"[""Chauncey Depew - the Director of the New Yor...",movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
2,tt0000502,1,nm0215752,actor,NaN,NaN,movie,Bohemios,Bohemios,0,1905,\N,100,\N
3,tt0000502,2,nm0252720,actor,NaN,NaN,movie,Bohemios,Bohemios,0,1905,\N,100,\N
4,tt0000574,2,nm0846894,actor,NaN,"[""School Master""]",movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475224,tt9916730,2,nm0059461,actor,NaN,NaN,movie,6 Gunn,6 Gunn,0,2017,\N,116,Drama
2475225,tt9916730,3,nm13233318,actor,NaN,"[""Vishal""]",movie,6 Gunn,6 Gunn,0,2017,\N,116,Drama
2475226,tt9916730,4,nm4852679,actor,NaN,NaN,movie,6 Gunn,6 Gunn,0,2017,\N,116,Drama
2475227,tt9916730,5,nm9050497,actor,NaN,NaN,movie,6 Gunn,6 Gunn,0,2017,\N,116,Drama


In [ ]:
# On compte le nombre d'occurence de chaque acteur dans ce nouveau df

df_vc = df_1['nconst'].value_counts()
df_vc

nconst
nm0103977     1133
nm0415549      661
nm0006982      626
nm0246703      621
nm0305182      617
              ... 
nm12321367       1
nm12321362       1
nm16704565       1
nm12205404       1
nm13233318       1
Name: count, Length: 782742, dtype: int64

In [ ]:
# On charge le df dans lequel le nom des acteurs est contenu et on le le merge avec notre df du nombre d'occurrence pour obtenir un acteur et son nb d'occurence

df_nom = pd.read_csv('BD/name.basics.tsv.gz', compression = 'gzip', sep = '\t')
df_nom.replace('\\N', pd.NA, inplace = True)
df_apparaition = pd.merge(df_vc,
                 df_nom,
                 how = 'left',
                 on = 'nconst') 

,nconst,count,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0103977,1133,Brahmanandam,1956,<NA>,"actor,music_department,soundtrack","tt1756476,tt3142764,tt12735488,tt5457772"
1,nm0415549,661,Jagathy Sreekumar,1951,<NA>,"actor,writer,music_department","tt0255306,tt0353010,tt0355633,tt0997173"
2,nm0006982,626,Adoor Bhasi,1929,1990,"actor,music_department,director","tt0268954,tt0156920,tt0076277,tt0268570"
3,nm0246703,621,Paquito Diaz,1937,2011,"actor,director,producer","tt0354726,tt0860529,tt0084125,tt0375567"
4,nm0305182,617,Eddie Garcia,1929,2019,"actor,director,assistant_director","tt0124988,tt2244376,tt0186511,tt0318412"
...,...,...,...,...,...,...,...
782737,nm12321367,1,Edgar Pinzón,<NA>,<NA>,actor,tt14056528
782738,nm12321362,1,Francisco Lozada,<NA>,<NA>,actor,tt14056528
782739,nm16704565,1,Martin Fernandez,<NA>,<NA>,actor,tt14056466
782740,nm12205404,1,M. Zaid,2007,<NA>,"actor,writer,director","tt13982558,tt14056318,tt14003522,tt13953632"


In [ ]:
# On passe la colonne startYear en float car on étair en object à cause du remplacement des cellules vides ( ce qui bloque pour min/max)

df_1['startYear'] = pd.to_numeric(df_1['startYear'])

In [ ]:
# On crée deux colonne une dans laquel on stock l'année minimum et l'autre max

df_min = df_1.groupby('nconst')['startYear'].min()
df_max = df_1.groupby('nconst')['startYear'].max()
df_2 = pd.merge(df_1,
                df_min,
                how = 'left',
                on = 'nconst') 
df_kpi1 = pd.merge(df_2,
                   df_max,
                   how = 'left',
                   on = 'nconst') 


In [ ]:
# Rename des colonnes pour y voir clair

df_kpi1 = df_kpi1.rename(columns = {'startYear_y' : 'minStartYear','startYear' : 'maxStartYear'})

,tconst,ordering,nconst,category,job,characters,titleType,primaryTitle,originalTitle,isAdult,startYear_x,endYear,runtimeMinutes,genres,minStartYear,maxStartYear
0,tt0000009,2,nm0183823,actor,NaN,"[""Mr. Hamilton""]",movie,Miss Jerry,Miss Jerry,0,1894.0,\N,45,Romance,1894.0,1930.0
1,tt0000009,3,nm1309758,actor,NaN,"[""Chauncey Depew - the Director of the New Yor...",movie,Miss Jerry,Miss Jerry,0,1894.0,\N,45,Romance,1894.0,1894.0
2,tt0000502,1,nm0215752,actor,NaN,NaN,movie,Bohemios,Bohemios,0,1905.0,\N,100,\N,1905.0,1905.0
3,tt0000502,2,nm0252720,actor,NaN,NaN,movie,Bohemios,Bohemios,0,1905.0,\N,100,\N,1905.0,1905.0
4,tt0000574,2,nm0846894,actor,NaN,"[""School Master""]",movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906.0,\N,70,"Action,Adventure,Biography",1906.0,1906.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475224,tt9916730,2,nm0059461,actor,NaN,NaN,movie,6 Gunn,6 Gunn,0,2017.0,\N,116,Drama,1989.0,2024.0
2475225,tt9916730,3,nm13233318,actor,NaN,"[""Vishal""]",movie,6 Gunn,6 Gunn,0,2017.0,\N,116,Drama,2017.0,2017.0
2475226,tt9916730,4,nm4852679,actor,NaN,NaN,movie,6 Gunn,6 Gunn,0,2017.0,\N,116,Drama,2011.0,2024.0
2475227,tt9916730,5,nm9050497,actor,NaN,NaN,movie,6 Gunn,6 Gunn,0,2017.0,\N,116,Drama,2017.0,2024.0


In [ ]:
# Colonne inutile

df_kpi1.drop(columns = 'startYear_x')

,tconst,ordering,nconst,category,job,characters,titleType,primaryTitle,originalTitle,isAdult,endYear,runtimeMinutes,genres,minStartYear,maxStartYear
0,tt0000009,2,nm0183823,actor,NaN,"[""Mr. Hamilton""]",movie,Miss Jerry,Miss Jerry,0,\N,45,Romance,1894.0,1930.0
1,tt0000009,3,nm1309758,actor,NaN,"[""Chauncey Depew - the Director of the New Yor...",movie,Miss Jerry,Miss Jerry,0,\N,45,Romance,1894.0,1894.0
2,tt0000502,1,nm0215752,actor,NaN,NaN,movie,Bohemios,Bohemios,0,\N,100,\N,1905.0,1905.0
3,tt0000502,2,nm0252720,actor,NaN,NaN,movie,Bohemios,Bohemios,0,\N,100,\N,1905.0,1905.0
4,tt0000574,2,nm0846894,actor,NaN,"[""School Master""]",movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,\N,70,"Action,Adventure,Biography",1906.0,1906.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475224,tt9916730,2,nm0059461,actor,NaN,NaN,movie,6 Gunn,6 Gunn,0,\N,116,Drama,1989.0,2024.0
2475225,tt9916730,3,nm13233318,actor,NaN,"[""Vishal""]",movie,6 Gunn,6 Gunn,0,\N,116,Drama,2017.0,2017.0
2475226,tt9916730,4,nm4852679,actor,NaN,NaN,movie,6 Gunn,6 Gunn,0,\N,116,Drama,2011.0,2024.0
2475227,tt9916730,5,nm9050497,actor,NaN,NaN,movie,6 Gunn,6 Gunn,0,\N,116,Drama,2017.0,2024.0


In [ ]:
# Merge pour récupérer tout ce qu'on a rechercher occurence min et max

df_kpi1_f = pd.merge(df_kpi1,
                     df_apparaition,
                     how = 'right',
                     on = 'nconst') 
df_kpi1_f

,tconst,ordering,nconst,category,job,characters,titleType,primaryTitle,originalTitle,isAdult,...,runtimeMinutes,genres,minStartYear,maxStartYear,count,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,tt0137154,5,nm0103977,actor,NaN,NaN,movie,Paradesi,Paradesi,0,...,\N,"Comedy,Drama,Romance",1987.0,2025.0,1133,Brahmanandam,1956,<NA>,"actor,music_department,soundtrack","tt1756476,tt3142764,tt12735488,tt5457772"
1,tt0155124,5,nm0103977,actor,NaN,NaN,movie,Rudra Veena,Rudra Veena,0,...,170,"Drama,Music",1987.0,2025.0,1133,Brahmanandam,1956,<NA>,"actor,music_department,soundtrack","tt1756476,tt3142764,tt12735488,tt5457772"
2,tt0177484,4,nm0103977,actor,NaN,NaN,movie,Jagadeka Veerudu Athiloka Sundari,Jagadeka Veerudu Athiloka Sundari,0,...,153,"Action,Comedy,Family",1987.0,2025.0,1133,Brahmanandam,1956,<NA>,"actor,music_department,soundtrack","tt1756476,tt3142764,tt12735488,tt5457772"
3,tt0187115,5,nm0103977,actor,NaN,"[""Store Manager""]",movie,Kshana Kshanam,Kshana Kshanam,0,...,158,"Action,Comedy,Crime",1987.0,2025.0,1133,Brahmanandam,1956,<NA>,"actor,music_department,soundtrack","tt1756476,tt3142764,tt12735488,tt5457772"
4,tt0187520,5,nm0103977,actor,NaN,"[""Hero friend""]",movie,S. P. Parasuram,S. P. Parasuram,0,...,140,"Action,Drama",1987.0,2025.0,1133,Brahmanandam,1956,<NA>,"actor,music_department,soundtrack","tt1756476,tt3142764,tt12735488,tt5457772"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475224,tt14056528,2,nm12321367,actor,NaN,"[""Matias""]",movie,En Caso de Emergencia,En Caso de Emergencia,0,...,61,Thriller,2007.0,2007.0,1,Edgar Pinzón,<NA>,<NA>,actor,tt14056528
2475225,tt14056528,1,nm12321362,actor,NaN,"[""Flavio""]",movie,En Caso de Emergencia,En Caso de Emergencia,0,...,61,Thriller,2007.0,2007.0,1,Francisco Lozada,<NA>,<NA>,actor,tt14056528
2475226,tt14056466,9,nm16704565,actor,NaN,"[""Masked Man""]",movie,Yudhra,Yudhra,0,...,142,"Action,Drama,Thriller",2024.0,2024.0,1,Martin Fernandez,<NA>,<NA>,actor,tt14056466
2475227,tt14056318,1,nm12205404,actor,NaN,"[""Muhammad Zaid""]",movie,7 Day Sunday,7 Day Sunday,0,...,\N,"Adventure,Family,Fantasy",NaN,NaN,1,M. Zaid,2007,<NA>,"actor,writer,director","tt13982558,tt14056318,tt14003522,tt13953632"


In [ ]:
# On drop les lignes en doubles et on affiche que celle qui nous intérresse 

df_kpi1_f.drop_duplicates(subset ="nconst", keep = 'first', inplace=True)
df_kpi1_f[['nconst', 'count', 'minStartYear', 'maxStartYear', 'primaryName', 'knownForTitles']]

,nconst,count,minStartYear,maxStartYear,primaryName,knownForTitles
0,nm0103977,1133,1987.0,2025.0,Brahmanandam,"tt1756476,tt3142764,tt12735488,tt5457772"
1133,nm0415549,661,1957.0,2023.0,Jagathy Sreekumar,"tt0255306,tt0353010,tt0355633,tt0997173"
1794,nm0006982,626,1953.0,1991.0,Adoor Bhasi,"tt0268954,tt0156920,tt0076277,tt0268570"
2420,nm0246703,621,1958.0,2005.0,Paquito Diaz,"tt0354726,tt0860529,tt0084125,tt0375567"
3041,nm0305182,617,1949.0,2019.0,Eddie Garcia,"tt0124988,tt2244376,tt0186511,tt0318412"
...,...,...,...,...,...,...
2475224,nm12321367,1,2007.0,2007.0,Edgar Pinzón,tt14056528
2475225,nm12321362,1,2007.0,2007.0,Francisco Lozada,tt14056528
2475226,nm16704565,1,2024.0,2024.0,Martin Fernandez,tt14056466
2475227,nm12205404,1,NaN,NaN,M. Zaid,"tt13982558,tt14056318,tt14003522,tt13953632"
